# Seminar 7 - Generalized Linear Models, Logistic Regression

## Logistic Regression

### Q1.1: Titanic Survival Dataset

In [41]:
titanic <- as.data.frame(Titanic)
titanic <- titanic[rep(1:nrow(titanic), titanic$Freq), -5]
head(titanic)

# library("alr4")
# library("dplyr")
# titanic <- TitanicSurvival |> rename(
#     Survived = survived,
#     Sex = sex,
#     Age = age,
#     Class = passengerClass)
# head(titanic)

,Class,Sex,Age,Survived
,<fct>,<fct>,<fct>,<fct>
3,3rd,Male,Child,No
3.1,3rd,Male,Child,No
3.2,3rd,Male,Child,No
3.3,3rd,Male,Child,No
3.4,3rd,Male,Child,No
3.5,3rd,Male,Child,No


a. Fit a logistic regression model with Sex, Age, and Class as explanatory variables.

In [42]:
glm(Survived ~ Age+Class+Sex, family="binomial", titanic)


Call:  glm(formula = Survived ~ Age + Class + Sex, family = "binomial", 
    data = titanic)

Coefficients:
(Intercept)     AgeAdult     Class2nd     Class3rd    ClassCrew    SexFemale  
     0.6853      -1.0615      -1.0181      -1.7778      -0.8577       2.4201  

Degrees of Freedom: 2200 Total (i.e. Null);  2195 Residual
Null Deviance:	    2769 
Residual Deviance: 2210 	AIC: 2222

b. Compare the odds of survival of the two genders. Which gender has a higher chance of survival? How many times higher?

In [44]:
odds <- function(df, sd) nrow(subset(df, sd)) / nrow(df)

titanic_men = subset(titanic, titanic$Sex == "Male")
odds(titanic_men, titanic_men$Survived == "Yes")

titanic_women = subset(titanic, titanic$Sex == "Female")
odds(titanic_women, titanic_women$Survived == "Yes")

[1] 0.2120162

[1] 0.7319149

In [45]:
odds(titanic_women, titanic_women$Survived == "Yes") /
odds(titanic_men, titanic_men$Survived == "Yes")

[1] 3.452165

c. Compute the odds of survival of each passenger class.

In [61]:
class_1 = subset(titanic, titanic$Class == "1st")
class_2 = subset(titanic, titanic$Class == "2nd")
class_3 = subset(titanic, titanic$Class == "3rd")
class_c = subset(titanic, titanic$Class == "Crew")

In [62]:
odds(class_1, class_1$Survived == "Yes")
odds(class_2, class_2$Survived == "Yes")
odds(class_3, class_3$Survived == "Yes")
odds(class_c, class_c$Survived == "Yes")

[1] 0.6246154

[1] 0.4140351

[1] 0.2521246

[1] 0.239548

e. Fit a logistic regression model with the same explanatory variables and two-way interaction
between Class and Sex.

In [49]:
m2 <- glm(Survived ~ Age+Class+Sex+Class:Sex, family="binomial", titanic)
summary(m2)


Call:
glm(formula = Survived ~ Age + Class + Sex + Class:Sex, family = "binomial", 
    data = titanic)

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)           0.3784     0.2734   1.384  0.16633    
AgeAdult             -1.0537     0.2304  -4.573 4.81e-06 ***
Class2nd             -1.2323     0.2688  -4.584 4.56e-06 ***
Class3rd             -1.0229     0.1991  -5.139 2.77e-07 ***
ClassCrew            -0.5746     0.1776  -3.235  0.00122 ** 
SexFemale             4.2331     0.5310   7.972 1.56e-15 ***
Class2nd:SexFemale   -0.4483     0.6460  -0.694  0.48772    
Class3rd:SexFemale   -2.8625     0.5633  -5.082 3.73e-07 ***
ClassCrew:SexFemale  -1.0862     0.8197  -1.325  0.18516    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2769.5  on 2200  degrees of freedom
Residual deviance: 2143.4  on 2192  degrees of freedom
AIC: 2161.4

Number of Fisher Scor

In [55]:
# # Find optimal decision threshold using 
# # Confusion matrix and statistics
# library("caret")
# th <- 0
# high <- -Inf
# for (t in c(0:1000)/1000) {
#     pred <- predict(m2, newdata = titanic, type='response')
#     pred <- factor(ifelse(pred > t, "yes", "no"))
#     cm <- confusionMatrix(data=pred, reference=titanic$Survived)
#     if (as.numeric(cm$overall["Accuracy"]) >= high) {
#         th <- t
#         high <- as.numeric(cm$overall["Accuracy"])
#     }
# }

# th

titanic_pr <- titanic
titanic_pr$Survived <- predict(m2, newdata = titanic, type='response') > 0.593

f. Use the analysis of deviance to see that the second model fits better than the first one. Is
the difference in fit significant?

g.  Based on the second model,
- Compute the difference in survival rates of two sexes.
- Using an appropriate odds ratio, compare the odds of survival of males in the first class with those of males in the third class.

In [56]:
titanic_pr_men = subset(titanic_pr, titanic_pr$Sex == "Male")
titanic_pr_women = subset(titanic_pr, titanic_pr$Sex == "Female")

odds(titanic_pr_men, titanic_pr_men$Survived == TRUE)
odds(titanic_pr_women, titanic_pr_women$Survived == TRUE)

[1] 0.002888504

[1] 0.6489362

In [57]:
class_1_men_pr = subset(titanic_pr_men, titanic_pr_men$Class == "1st")
class_3_men_pr = subset(titanic_pr_men, titanic_pr_men$Class == "3rd")

odds(class_1_men_pr, class_1_men_pr$Survived == TRUE)
odds(class_3_men_pr, class_3_men_pr$Survived == TRUE)

[1] 0.02777778

[1] 0

### Q1.2: Anesthesia Surgery Dataset

In [71]:
surgery <- data.frame(
    type = c(0,0,0,1,1,1,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1),
    sore = c(0,0,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,1,0,1,0,0,1,0,1,0,1,1,0,1,0,0),
    duration = c(45,15,40,83,90,25,35,65,95,35,75,45,50,75,30,25,20,60,70,30,60,61,
                 65,15,20,45,15,25,15,30,40,15,135,20,40)
)
head(surgery)

,type,sore,duration
,<dbl>,<dbl>,<dbl>
1,0,0,45
2,0,0,15
3,0,1,40
4,1,1,83
5,1,1,90
6,1,1,25


a. Disregarding the duration information, compute the observed joint distribution of sore throat
incidence and type of anesthesia and carry out a chi-squared test to see whether these two
variables are independent from each other.

In [72]:
chisq.test(surgery$type, surgery$sore)


	Pearson's Chi-squared test with Yates' continuity correction

data:  surgery$type and surgery$sore
X-squared = 2.3405, df = 1, p-value = 0.1261


b. Fit a logistic regression model to investigate whether the surgery duration and anesthesia
type have a significant effect on the occurrence of sore throat.

In [73]:
m3 <- glm(sore ~ type + duration, family="binomial", surgery)

c. Which statistical test can be used to assess whether the effect of duration differs for the two types of anesthesia?

## 2. Multiple linear regression

### Q2.1: Prostate Data

In [325]:
df = read.table("data/prostate.dat")
head(df)

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<lgl>
1,-0.5798185,2.769459,50,-1.386294,0,-1.386294,6,0,-0.4307829,TRUE
2,-0.9942523,3.319626,58,-1.386294,0,-1.386294,6,0,-0.1625189,TRUE
3,-0.5108256,2.691243,74,-1.386294,0,-1.386294,7,20,-0.1625189,TRUE
4,-1.2039728,3.282789,58,-1.386294,0,-1.386294,6,0,-0.1625189,TRUE
5,0.7514161,3.432373,62,-1.386294,0,-1.386294,6,0,0.3715636,TRUE
6,-1.0498221,3.228826,50,-1.386294,0,-1.386294,6,0,0.7654678,TRUE


b. Compute the correlation matrix of the data and a scatterplot matrix of pairwise plots (omit the dummy variable called train).

## Practice Test

### Question 1
Refer to question Q1.1 of the GLM seminar handout (Titanic Survival data). 
Fit a logistic regression model with age, sex, passenger class, and interaction of the latter two as predictors and survival as response. Compute the estimated survival rate of the third class passengers.

This rate, expressed as percentage rounded to the closest integer, is equal to ??? .

In [65]:
round(odds(class_3, class_3$Survived == "Yes") * 100)

[1] 25

### Question 2
Refer to question Q1.1 of the GLM seminar handout (Titanic survival data). 
Fit a logistic regression model with age, sex, passenger class, and interaction of the latter two as predictors and survival as response.  Based on this model,   estimate the ratio of the odds of survival of female passengers in the 1st class and odds of survival of female passengers in the 3rd class. 

This ratio, rounded to the closest integer, is equal to ??? .

In [79]:
class_1_women = subset(titanic_women, titanic_women$Class == "1st")
class_3_women = subset(titanic_women, titanic_women$Class == "3rd")
# Actual
round(
    odds(class_1_women, class_1_women$Survived == "Yes")/
    odds(class_3_women, class_3_women$Survived == "Yes"))

class_1_women_pr = subset(titanic_pr_women, titanic_pr_women$Class == "1st")
class_3_women_pr = subset(titanic_pr_women, titanic_pr_women$Class == "3rd")
# Predicted
round(
    odds(class_1_women_pr, class_1_women_pr$Survived == TRUE)/
    odds(class_3_women_pr, class_3_women_pr$Survived == TRUE))

# Calculated
round(exp(-0.1639) / exp(-2.8625))

[1] 2

[1] 6

[1] 15

### Question 3
Fit two logistic regression models:  a model with and a model without the interaction between surgery duration and anesthesia type.

Perform the likelihood-ratio test to see whether the interaction effect is significant. Rounded to two decimal places, the p-value for this test is equal to 

In [334]:
library("lmtest")

without <- glm(sore ~ type + duration, data=surgery, family="binomial")
with <- glm(sore ~ type + duration + type:duration, data=surgery, family="binomial")

lrtest(with, without)
logLik(with)
logLik(without)
teststat <- -2 * (as.numeric(logLik(without))-as.numeric(logLik(with)))

round(pchisq(teststat, df = 1, lower.tail = FALSE), 2)

,#Df,LogLik,Df,Chisq,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4,-14.16053,NA,NA,NA
2,3,-15.06897,-1,1.816886,0.1776844


'log Lik.' -14.16053 (df=4)

'log Lik.' -15.06897 (df=3)

[1] 0.18